In [ ]:
# This script installs the necessary Python packages for data analysis and visualization.
!pip install pandas numpy matplotlib seaborn missingno scipy

In [22]:
# Import all necessary libraries and data

%pip install openpyxl

import pandas as pd
import glob
import re  # For extracting year from filename

# Initialize storage
combined_data = []

# Get all Excel files
file_paths = glob.glob("C:/Users/Lenovo/Desktop/Dissertaion/China Data/Sales/*.xlsx")  # Update path

for file in file_paths:
    # Extract year from filename (assuming format like "...2023.xlsx")
    year = re.search(r'\d{4}', file).group()  # Finds first 4-digit number
    
    # Read all sheets from current file
    sheets_dict = pd.read_excel(file, sheet_name=None)
    
    for sheet_name, df in sheets_dict.items():
        # Add identifier columns
        df['year'] = int(year)          # From filename
        df['type'] = sheet_name         # From sheet name
        
        combined_data.append(df)

# Combine all DataFrames
final_df = pd.concat(combined_data, ignore_index=True)

# Import Price Data
price_file = pd.read_excel(
	"C:/Users/Lenovo/Desktop/Dissertaion/China Data/Price 2015-202309.xlsx",
	sheet_name="中国汽车分车型每月销售量"
)


Note: you may need to restart the kernel to use updated packages.


In [55]:
# Process Price Data

# Define a function to convert price ranges to midpoints
def price_range_to_midpoint(price_str):
    try:
        # Split the string by hyphen
        price_parts = price_str.split('-')
        
        # Convert each part to float
        low = float(price_parts[0])
        high = float(price_parts[1])
        
        # Calculate and return midpoint
        return (low + high) / 2
    except:
        # Return None or some default value if conversion fails
        return None

# Calculate midpoints for price ranges
price_file['price_midpoint'] = price_file['售价（万元）'].apply(price_range_to_midpoint)

# Calculate yearly sales-weighted price for each model
# 1. calculate total sales and sales-weighted price sum by model
yearly_data = price_file.groupby('车型').apply(
    lambda x: pd.Series({
        'Total_Sales': x['销量'].sum(),
        'Weighted_Price_Sum': (x['销量'] * x['price_midpoint']).sum()
    })
)

# Then calculate the weighted average price
yearly_data['Weighted_Avg_Price'] = yearly_data['Weighted_Price_Sum'] / yearly_data['Total_Sales']

# 2. Reset index to make Model a column again
yearly_data = yearly_data.reset_index()

# 3. Check yearly_data structure
print(yearly_data.head())

# 4. Save results to a new Excel file
yearly_data.to_excel('yearly_weighted_prices.xlsx', index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_212\1767318256.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  yearly_data = price_file.groupby('车型').apply(


        车型  Total_Sales  Weighted_Price_Sum  Weighted_Avg_Price
0      212       2470.0           25416.300              10.290
1   AION S     445125.0         7113097.500              15.980
2   AION V      67839.0         1457520.915              21.485
3   AION Y     310087.0         4800146.760              15.480
4  Aion LX       9099.0          344033.190              37.810


In [41]:
# View variable names
print(final_df.columns)

# Translate variable names
# Manual translation
translation_map = {
    '省份': 'province',
    '品牌': 'brand',
    '车型': 'model',
    '燃料类型': 'fuel_type',
    '功率': 'power',
    '销量'  : 'sales',
    '总质量': 'mass',
}

# Apply translations
combined_df = final_df.rename(columns=translation_map)


# Move 'year' and 'type' to the front
cols = ['year', 'type'] + [col for col in combined_df.columns if col not in ['year', 'type']]
sales = combined_df[cols]

print(sales.columns)

# Converts data types
sales = sales.convert_dtypes()

# Convert specific columns
df[['mass', 'power']] = df[['mass', 'power']].astype('Int64')

sales.dtypes

Index(['省份', '品牌', '车型', '燃料类型', '总质量', '功率', '销量', 'year', 'type'], dtype='object')
Index(['year', 'type', 'province', 'brand', 'model', 'fuel_type', 'mass',
       'power', 'sales'],
      dtype='object')


KeyError: "None of [Index(['mass', 'power'], dtype='object')] are in the [columns]"